In [1]:
!pip install streamlit pyngrok --quiet
import torch
import textwrap
from transformers import BertForQuestionAnswering,BertTokenizer
import json
import firebase_admin
from firebase_admin import credentials, db

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00


In [6]:
#global variables
data = None
abstract = None
model = None
tokenizer = None
firebase_initialization = False

In [8]:
## run it only once
##retrieving data.json only once throughout the streamlit app
def initialize():
  ##initializing firebase instace based on condition
  if(firebase_initialization==False):
    cred_obj = credentials.Certificate("/content/naanbert-firebase-adminsdk-k1t8m-65d91682f0.json")

    default_app = firebase_admin.initialize_app(cred_obj, {
    'databaseURL': 'https://naanbert-default-rtdb.firebaseio.com/'
    })

  #initializing models as well
  model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
  tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [10]:
initialize() #run only once every time you connect to a hosted runtime/or even locally

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def get_context(key):
  ref = db.reference('/')
  data = ref.get()
  abstract = data[key]

get_context("Introduction")

In [23]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    question = "How many parameters does BERT-large have?"
    answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')




In [24]:
result = answer_question("What is nabh?",abstract)
print(result)

AttributeError: 'NoneType' object has no attribute 'encode'

In [ ]:
%%writefile app.py
import streamlit as st
def main():
  st.title("app")

  user_input = st.text_input("You:", "")

  # Button to submit the input and get a response
  if st.button("Send"):
      if user_input:
          # Get the chatbot's response
          response = answer_question(user_input, abstract)

          # Display the response
          st.text_area("Chatbot:", value=response, height=100)

if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
from pyngrok import ngrok
ngrok.kill()
!ngrok config add-authtoken 2l4bCHXzM5Ci2p2lsM3fAi7ymkk_3jzBYpRfq2Hz1RFQWbeFp

# Run Streamlit app and ngrok tunnel
import subprocess
process = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501'])
public_url = ngrok.connect(8501).public_url
print(f"Streamlit App is running at: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit App is running at: https://2fed-35-221-211-10.ngrok-free.app
